In [22]:
ann_str_to_num  = {
    '(AFIB\x00': 0,
    '(ASYS\x00': 1,
    '(B\x00': 2,
    '(BI\x00': 3,
    '(HGEA\x00': 4,
    '(N\x00': 5,
    '(NSR\x00': 5,
    '(NOD\x00': 6,
    '(NOISE\x00': 7,
    '(PM\x00': 8,
    '(SBR\x00': 9,
    '(SVTA\x00': 10,
    '(VER\x00': 11,
    '(VF\x00': 12,
    '(VFIB\x00': 12,
    '(VFL\x00': 13,
    '(VT\x00': 14
}

ann_num_to_str = {}
for k, v in ann_str_to_num.items():
    if v == 5:
        k = '(N\x00'
    elif v == 12:
        k = '(VF\x00'
    ann_num_to_str[v] = k
for item in ann_num_to_str.items():
    print item

(0, '(AFIB\x00')
(1, '(ASYS\x00')
(2, '(B\x00')
(3, '(BI\x00')
(4, '(HGEA\x00')
(5, '(N\x00')
(6, '(NOD\x00')
(7, '(NOISE\x00')
(8, '(PM\x00')
(9, '(SBR\x00')
(10, '(SVTA\x00')
(11, '(VER\x00')
(12, '(VF\x00')
(13, '(VFL\x00')
(14, '(VT\x00')


In [2]:
import wfdb
import vfdb_annotate_r as my_vfdb

record_id = '418'
qrs_i = my_vfdb.load_qrs_i(record_id)

qrs_amp = my_vfdb.load_qrs_amp(record_id)
ann = my_vfdb.get_annotation(record_id)
annot_positions = my_vfdb.get_annotation_positions(ann)
annotated_r_peaks = my_vfdb.create_qrs_annotation_mapping(qrs_i, qrs_amp, annot_positions)
annotated_rr_segments = my_vfdb.create_rr_annotation(annotated_r_peaks)

print annotated_rr_segments[:,0].size

4300


In [4]:
print vfdb_annotate_r.VFDB_RECORDS

[418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 602, 605, 607, 609, 610, 611, 612, 614, 615]


In [20]:
import numpy
import vfdb_annotate_r as my_vfdb
print "\nrecord \t num_of_RR"
for record in my_vfdb.VFDB_RECORDS:
    filepath = my_vfdb.QRS_LOCATION + str(record) + '_RR.csv'
    rr = numpy.genfromtxt(filepath, delimiter=',')
    print record, '\t', rr[:,0].size


record 	 num_of_RR
418 	4300
419 	3905
420 	2923
421 	4525
422 	4072
423 	3537
424 	3795
425 	2128
426 	3774
427 	3894
428 	3265
429 	3613
430 	4695
602 	3307
605 	1921
607 	1847
609 	2214
610 	2166
611 	3947
612 	2489
614 	3329
615 	1592


In [10]:
# Find all different labels in every annotation
import pprint

records = [str(x) for x in my_vfdb.VFDB_RECORDS]
for record in records:
    ann = my_vfdb.get_annotation(record)
    #print record
    #pprint.pprint(set(ann.aux_note))
    labels = set(ann.aux_note)
    print record, labels

418 set(['(N\x00', '(VFL\x00'])
419 set(['(N\x00', '(VFL\x00', '(NOISE\x00'])
420 set(['(N\x00', '(NOISE\x00', '(VT\x00'])
421 set(['(N\x00', '(NOISE\x00', '(VT\x00'])
422 set(['(N\x00', '(NOISE\x00', '(VT\x00', '(VFIB\x00'])
423 set(['(N\x00', '(ASYS\x00', '(NOD\x00', '(NOISE\x00', '(VT\x00', '(AFIB\x00'])
424 set(['(N\x00', '(ASYS\x00', '(NSR\x00', '(VFIB\x00', '(NOD\x00', '(NOISE\x00', '(VFL\x00'])
425 set(['(N\x00', '(NOISE\x00', '(VT\x00', '(B\x00'])
426 set(['(VF\x00', '(N\x00', '(SVTA\x00', '(NOISE\x00', '(VT\x00'])
427 set(['(N\x00', '(ASYS\x00', '(VT\x00'])
428 set(['(BI\x00', '(NOISE\x00', '(VT\x00'])
429 set(['(BI\x00', '(NOISE\x00', '(VT\x00', '(VFL\x00'])
430 set(['(VF\x00', '(HGEA\x00', '(ASYS\x00', '(NOISE\x00', '(BI\x00', '(SBR\x00', '(VER\x00', '(VFL\x00', '(VT\x00'])
602 set(['(N\x00', '(PM\x00', '(ASYS\x00', '(VT\x00', '(NOISE\x00'])
605 set(['(NOISE\x00', '(VT\x00'])
607 set(['(SVTA\x00', '(NOD\x00', '(ASYS\x00', '(VT\x00'])
609 set(['(N\x00', '(HGEA\x00', '(VT\x00'

In [42]:
from collections import Counter
import numpy
import pprint
import vfdb_annotate_r as my_vfdb

records = [str(x) for x in my_vfdb.VFDB_RECORDS]
#records = ['418']
# each segment will have 100 RR
SEGMENT_LEN = 100

for record in records:
    filepath = my_vfdb.QRS_LOCATION + str(record) + '_RR.csv'
    rrs = numpy.genfromtxt(filepath, delimiter=',')
    rows, columns = rrs.shape
    num_segments = rows / (SEGMENT_LEN / 2)
    i = 0
    l = []
    while i < num_segments:
        start = i * (SEGMENT_LEN / 2)
        if i == num_segments - 1:
            end = rows - 1
        else:
            end = start + SEGMENT_LEN
        # start is the idx of the first RR of this segment
        # end is the idx of the last RR in this segment
        labels = rrs[start:end, 3]
        data = Counter(labels)
        most_frequent = max(data)
        if data[most_frequent] >= 0.8 * SEGMENT_LEN:
            l.append(ann_num_to_str[most_frequent])
        else:
            l.append("NO_LABEL")
                
        #print [ann_num_to_str[x] for x in list(set(labels))]            
        i += 1
    print "record", record, "has", num_segments, "segments", Counter(l)

record 418 has 86 segments Counter({'NO_LABEL': 49, '(N\x00': 37})
record 419 has 78 segments Counter({'(VFL\x00': 34, '(N\x00': 26, 'NO_LABEL': 18})
record 420 has 58 segments Counter({'(N\x00': 31, '(VT\x00': 25, 'NO_LABEL': 2})
record 421 has 90 segments Counter({'(N\x00': 57, '(VT\x00': 19, 'NO_LABEL': 14})
record 422 has 81 segments Counter({'(N\x00': 44, '(VF\x00': 24, '(VT\x00': 10, 'NO_LABEL': 3})
record 423 has 70 segments Counter({'(N\x00': 23, '(VT\x00': 23, '(AFIB\x00': 19, 'NO_LABEL': 5})
record 424 has 75 segments Counter({'(N\x00': 58, 'NO_LABEL': 10, '(VF\x00': 5, '(VFL\x00': 2})
record 425 has 42 segments Counter({'(N\x00': 27, 'NO_LABEL': 8, '(B\x00': 5, '(VT\x00': 2})
record 426 has 75 segments Counter({'(VF\x00': 42, '(N\x00': 15, 'NO_LABEL': 14, '(VT\x00': 4})
record 427 has 77 segments Counter({'(VT\x00': 44, '(ASYS\x00': 16, '(N\x00': 15, 'NO_LABEL': 2})
record 428 has 65 segments Counter({'(BI\x00': 56, 'NO_LABEL': 7, '(VT\x00': 2})
record 429 has 72 segments Co